# Python in genomic analysis - a case study

1. Read the paper (Optional if you are looking at your own data)
2. Formulate the question
3. Find the data
4. Obtain, process
5. Analyze!

## Paper: Hormone-Responsive Enhancer-Activity Maps Reveal Predictive Motifs, Indirect Repression, and Targeting of Closed Chromatin

### Questions: 
1. Where are the ecdysone-induced enhancers in chromosome 3L?
2. What transcription factors may bind at these enhancers?




### How to get the data?

#### This happens behind the scenes for this class to save time, but I wanted to share the whole process so you are aware of how you could do this by yourself.

1. Get the "reads" as fastq files<br><br>
`
fastq-dump SRR1204764
fastq-dump SRR1204762
`
<br><br>
2. Align the reads to the genome<br><br>
`
bowtie2 -x /beevol/home/srinivas/lib/bowtie2/dm6/genome -q SRR1204764.fastq --no-unal -p 10 >  w_ecd.sam
bowtie2 -x /beevol/home/srinivas/lib/bowtie2/dm6/genome -q SRR1204762.fastq --no-unal -p 10 >  wo_ecd.sam
`
<br><br>
3. Convert the sam to bam (binary compressed format that will be read by bedtools)<br><br>
`
samtools view -Sb w_ecd.sam > w_ecd.bam
samtools view -Sb wo_ecd.sam > wo_ecd.bam
`
<br><br>

4. Convert bam to bed (bed file will have one read per line, a human readable form)<br><br>
`
bedtools bamtobed -i w_ecd.bam > w_ecd.bed
bedtools bamtobed -i wo_ecd.bam > wo_ecd.bed
`
<br><br>

5. Convert bed to wig (we collapse the reads into a wig file)<br><br>

`123456789`<br>
`----_____`<br>
`___----__`<br>
`_____----`<br>
`______---`<br>
`______---`<br>
`111212433`<br>

Let us start by reading in wiggle files, which have the STARR-seq read densities for every 10 bp in chromosome 3L.

In [ ]:
def readwig (inFile):
  wig=open(inFile, 'r' )
  val={}
  for line in wig:
    lineL = line.split()
    if "chr" in lineL[1]:
      cj=lineL[1]
      chrom=cj[9:]
      print(chrom)
      val.setdefault(chrom,{})
    elif "rack" not in line:
      pos=int(lineL[0])
      val[chrom][pos] = float(lineL[1])
  return(val)

In [ ]:
w_ecd = readwig('3L.w_ecd.wig')
wo_ecd = readwig('3L.wo_ecd.wig')

### 1. Directly view the data
How does a region on the chromosome 3L look in our datasets? Since this experiment is mapping enhancers activated after ecdysone treatment, let us look at a gene that is responsive to ecdysone: Eip75B, which goes from 17950953 to 18064696 on chromosome 3L.

In [ ]:
import numpy as np
subset_w_ecd = []
for i in range(17950953,18064696):
    if i in w_ecd['3L']:
        subset_w_ecd.append(w_ecd['3L'][i])
ns_w_ecd = np.array(subset_w_ecd)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(ns_w_ecd)
plt.show()

### 2. How do we extract peaks from STARR-seq wig files?

The peaks in STARR-seq read density would correspond to enhancers, which we want to identify.

SciPy has an in-built peak caller, which we can use. We will first use some toy data to understand how the peak caller works. 

In [ ]:
import scipy
from scipy.signal import find_peaks
from numpy import pi

x = np.linspace( 0, 6*pi, 600 )
f = np.sin(x)

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html

peaks, _ = find_peaks(f, height=0)

In [ ]:
plt.plot(x,f)
plt.plot(x[peaks], f[peaks], "ro")
plt.ylim(-1.1,1.1)
plt.show()

Now let us try the peak caller on our ecdysone-responsive gene.

In [ ]:
indices = np.arange(0,len(ns_w_ecd))
peaks, _ = find_peaks(ns_w_ecd) 
plt.plot(ns_w_ecd)
plt.plot(indices[peaks],ns_w_ecd[peaks], "ro")
plt.show()

That is too many peaks! That is because find_peaks is identifying every local maxima. We can set thresholds to cull out most of the non-peaks. What if we want to find local maxima that are at least 4 standard deviations away from the mean.

In [ ]:
peaks, _ = find_peaks(ns_w_ecd, height=4*ns_w_ecd.std()+ns_w_ecd.mean())
plt.plot(ns_w_ecd)
plt.plot(indices[peaks],ns_w_ecd[peaks], "ro")
plt.show()
#our unit is 10 bp, so distance of 10 is highest peak within 100 bp

If you think the distribution is too rough, we can use SciPy functions to smooth the distribution. For example, we can try the Savitsky Golay filter. Let us try to smooth this distribution. We will use Savitsky Golay filter.
<br>
https://en.wikipedia.org/wiki/Savitzky%E2%80%93Golay_filter
<br>
https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.signal.savgol_filter.html

In [ ]:
from scipy.signal import savgol_filter
smooth_ar = savgol_filter(ns_w_ecd,9,1)
peaks, _ = find_peaks(smooth_ar, height=4*smooth_ar.std()+smooth_ar.mean(), distance=10)
plt.plot(smooth_ar,)
plt.plot(indices[peaks],smooth_ar[peaks], "ro")
plt.show()

### 3. Call peaks on the whole chromosome
We will smooth the whole chromosome and call all peaks. 

In [ ]:
#let us apply this to the whole chromosome

temp_list = [] #Values
pos_list = [] #Chromosome positions

for i in sorted(w_ecd['3L'].keys()):
    temp_list.append(w_ecd['3L'][i])
    pos_list.append(i)

chrom_array = np.array(temp_list)

smooth_ar = savgol_filter(chrom_array,9,1)
peaks_ecd, _ = find_peaks(smooth_ar, height=4*smooth_ar.std()+smooth_ar.mean(), distance=10)

How many peaks did we find?

In [ ]:
len(peaks_ecd)

Let us perform the same analysis on the STARR-seq wig file from cells not exposed to ecdysone. These will be our control enhancers

In [ ]:
#Now for the non ecdysone treated sample

temp_list = []
pos_list_wo = []

for i in sorted(wo_ecd['3L'].keys()):
    temp_list.append(wo_ecd['3L'][i])
    pos_list_wo.append(i)

chrom_array_wo = np.array(temp_list)

smooth_ar_wo = savgol_filter(chrom_array_wo,9,1)
peaks_wo_ecd, _ = find_peaks(smooth_ar_wo, height=4*smooth_ar_wo.std()+smooth_ar_wo.mean(), distance=10)

#### Do the peaks make sense?

Let us first understand how to trace back the genomic location from the peak<br><br>

`Indices  :  0  1  2  3  4  5  6  7  8`<br>
`Positions: 22 23 24 25 26 27 28 29 30`<br>
`maxima   :           *        *      `<br><br>
`peak_list : 3,6`<br><br>
`peak_list[0] = 3`<br><br>
`positions[peak_list[0]] = 25`
<br><br>Now we can ask if the signal around the peak position is high.


Let us spit out a random peak from the list.
1. Find a random number between 0 and the number of peaks idenfified
2. Find the chromosomal position of the peak that correspond to the random  peak index identified in the step above.

In [ ]:
rand_index = int(np.random.rand()*len(peaks_ecd)-1)
print(rand_index)
print(peaks_ecd[rand_index])
peak_val = pos_list[peaks_ecd[rand_index]]
print(peak_val)

We will plot the STARR-seq density around this random peak and ask if it looks like a peak.

In [ ]:
subset_w_ecd = []
location = []
for i in range(peak_val-2000,peak_val+2000):
    if i in w_ecd['3L']:
        subset_w_ecd.append(w_ecd['3L'][i])
    else:
        subset_w_ecd.append(0)
    location.append(i-peak_val)
ns_w_ecd = np.array(subset_w_ecd)
plt.plot(location,ns_w_ecd)
plt.show()

### I don't want to see one peak at a time. How about an average profile of all peaks?

`
position: -2000 -1990 ....... 0  ....... 1990 2000
peak 1  :    x1    x2 ....... xn .......   xm   xp
peak 2  :    y1    y2 ....... yn .......   ym   yp
.
.
.
.`<br>
`peak j  :    z1    z2 ....... zn .......   zm   zp`<br>
`--------------------------------------------------`<br>
`Average :    a1    a2 ....... an .......   am   ap`<br>

In [ ]:
import pandas as pd

peak_score_w_ecd = {}

fh = open("py_peaks_w_ecd.bed","w")
ave_vals = {}
location = []
for p in peaks_ecd:
    peak_pos = pos_list[p]
    temp_str = "chr3L\t" + str(peak_pos-250) + "\t" + str(peak_pos+250) + "\t" + str(p) + "\n"
    fh.write(temp_str)
    for i in range(peak_pos-2000,peak_pos+2000):
        if i in w_ecd['3L']:
            ave_vals.setdefault('val',{})
            if i-peak_val in ave_vals:
                ave_vals['val'][i-peak_pos] += w_ecd['3L'][i]
            else:
                ave_vals['val'][i-peak_pos] = w_ecd['3L'][i]
    score=0
    key="chr3L:" + str(peak_pos-250) + "-" + str(peak_pos+250)
    for i in range(peak_pos-100,peak_pos+100):
        if i in w_ecd['3L']:
            score+=w_ecd['3L'][i]
        peak_score_w_ecd[key] = score/20

fh.close()
df_ave_vals = pd.DataFrame(ave_vals)
df_ave_vals['val'] /= len(peaks_ecd)
#df_ave_vals.head()
plt.plot(df_ave_vals)
plt.show()

In [ ]:
fh = open("py_peaks_wo_ecd.bed","w")
ave_vals_wo = {}
location_wo = []
for p in peaks_wo_ecd:
    peak_pos = pos_list_wo[p]
    temp_str = "chr3L\t" + str(peak_pos-250) + "\t" + str(peak_pos+250) + "\t" + str(p) + "\n"
    fh.write(temp_str)
    for i in range(peak_pos-2000,peak_pos+2000):
        if i in wo_ecd['3L']:
            ave_vals_wo.setdefault('val',{})
            if i-peak_val in ave_vals_wo:
                ave_vals_wo['val'][i-peak_pos] += wo_ecd['3L'][i]
            else:
                ave_vals_wo['val'][i-peak_pos]  = wo_ecd['3L'][i]

fh.close()
df_ave_vals = pd.DataFrame(ave_vals_wo)
df_ave_vals['val'] /= len(peaks_wo_ecd)
plt.plot(df_ave_vals)
plt.show()

## 4. What Transcription Factor motifs underlie enhancer peaks?

Now that we have identified the peaks in STARR-seq densitites that correspond to enhancers, let us now find the transcription factor motifs that are enriched in the enhancers. We will first extract the sequences underlying the enhancers that are activated in the presence of ecdysone. We will use bedtools to: 
1. Identify enhancers unique to ecdysone response
2. Extract sequences of the enhancers that respond to ecdysone and control enhancers that don't. 

In [ ]:
import pybedtools 

enhancers_w_ecd = pybedtools.BedTool('py_peaks_w_ecd.bed')
enhancers_wo_ecd = pybedtools.BedTool('py_peaks_wo_ecd.bed')

#Enhancers unique to Ecdysone treatment:
activated_enhancers = enhancers_w_ecd - enhancers_wo_ecd 

print(activated_enhancers.count())

g = activated_enhancers.sequence(fi='3L.fa')

#g.seqfn

fo = open('w_ecd_peaks.fa', 'w')
fo.write(open(g.seqfn).read())

g = enhancers_wo_ecd.sequence(fi='3L.fa')

fo = open('wo_ecd_peaks.fa', 'w')
fo.write(open(g.seqfn).read())

I identified motifs enriched in the enhancer sequence using FIMO, using all Drosophila enhancers in JASPAR (http://jaspar.genereg.net/search?q=&collection=CORE&tax_group=insects). We will parse the fimo output file using pandas.

In [ ]:
w_ecd_motifs = pd.read_csv('fimo_w_ecd.txt',sep='\t')
w_ecd_motifs.head()

In [ ]:
wo_ecd_motifs = pd.read_csv('fimo_wo_ecd.txt',sep='\t')
wo_ecd_motifs.head()

## 5. What motifs are enriched in the ecdysone receptors?

1. Let us count the number of motifs in each set of enhancers using the groupby method in pandas.
2. Normalize the count of each motif to the total number of motif occurence.

In [ ]:
norm_w_ecd = (w_ecd_motifs.groupby('motif_alt_id').count()/len(w_ecd_motifs)).iloc[:,0:1]
norm_wo_ecd = (wo_ecd_motifs.groupby('motif_alt_id').count()/len(wo_ecd_motifs)).iloc[:,0:1]
print(norm_w_ecd.head())
print(norm_wo_ecd.head())
mergeddf = pd.merge(norm_w_ecd, norm_wo_ecd, how = 'left', on = 'motif_alt_id')
mergeddf.head()

The ratio of normalized number of occurences of each motif in enhancers that respond to ecdysone compared to control enhancers gives us an idea of  the transcription factors that may be responding to ecdysone.

In [ ]:
ratio_w  = mergeddf['# motif_id_x']
ratio_wo = mergeddf['# motif_id_y']

Enrichment = (ratio_w / ratio_wo)
#print(Enrichment)

#Now add these values as a new column
newdf = mergeddf.assign(Enrichment = Enrichment)
print(newdf.sort_values(by='Enrichment',ascending=False).head(10))

the top two transcription factor motifs enriched in response to ecdysone are 

1. Broad-complex core protein isoforms 1/2/3/4/5 - transcription factors that are activated by ecdysone
2. Ecdysone receptor

So the top two motifs in the enhancers activated by ecdysone correspond to transcription factors that respond to ecdysone - this makes a lot of sense. But to confirm our finding, let us ask if the STARR-seq densities at the enhancers with these motifs are high. We can extract the STARR-seq density at these enhancers using the dictionary peak_score_w_ecd we created when we were making average plots. We designed the keys of this dictionary to match the "sequence name" in the FIMO output file. So we can match every motif occurence in FIMO with a corresponding enhancer score. We can demonstrate this below, where we are using the "sequence_name" column of the first row to extract a string that can be a key in the peak_score_w_ecd dictionary.

In [ ]:
peak_score_w_ecd[w_ecd_motifs.iloc[0]['sequence_name']]

Let us now extract the scores of the enhancers for each motif occurence in the FIMO output.

In [ ]:
with_score = pd.DataFrame()
for i in newdf.sort_values(by='Enrichment',ascending=False).head(5).index.values:
    new_df = w_ecd_motifs.query("motif_alt_id == '{0}'".format(i)).iloc[:,1:3]
    #print(new_df.head(1))
    score_col = []
    for index, row in new_df.iterrows():
        score_col.append(peak_score_w_ecd[row['sequence_name']])
    new_df = new_df.assign(score_col = score_col)
    with_score = with_score.append(new_df)
with_score.head()

How does the STARR-seq score look for enhancers with different motifs? Let us make a boxplot of the scores for each motif. Since we made a dataframe with the scores for each motif occurence, we can use seaborn to make a boxplot easily. We see that enhancers with ecdysone receptor motif have higher score than all others.

In [ ]:
import seaborn as sns
sns.boxplot(data = with_score, x = 'motif_alt_id', y = 'score_col')
plt.show()